In [1]:
using Pkg
Pkg.add("Agents")
Pkg.add("DataStructures")

    Updating registry at `/opt/julia/registries/General.toml`
   Resolving package versions...
  No Changes to `/opt/julia/environments/v1.8/Project.toml`
  No Changes to `/opt/julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `/opt/julia/environments/v1.8/Project.toml`
  No Changes to `/opt/julia/environments/v1.8/Manifest.toml`


In [2]:
using Agents
using Random
using DataStructures

In [3]:
@agent AntAgent GridAgent{2} begin
    colony::Int
    has_food::Bool
    path_home::Stack{Tuple}
    facing_direction::Tuple # tuple of the direction the ant is facing - 
        # (1, 0) - right/east
        # (-1, 0) - left/west
        # (0, 1) - down/south
        # (0, -1) - up/north
    food_returned_nest::Int
    food_picked_up::Int
    steps::Int
end

In [4]:
function initialize_ants(;dimensions = (64, 64), number_nests = 1, number_ants = 50, number_food = 3, max_food = 50, random_seed = 42, diffusion_rate = 10, ant_pheremone_amount = 10)
    rng = Random.Xoshiro(random_seed)
    
    nest_locations = Dict()
    pheremone_trails = Dict()
    food_collected = Dict()
    for nest_counter in 1:number_nests
        x = rand(1:dimensions[1])
        y = rand(1:dimensions[2])
        nest_locations[nest_counter] = (x, y)
        
        pheremone_trails[nest_counter] = zeros(Int, dimensions)
        food_collected[nest_counter] = 0
    end
    
    food_locations = falses(dimensions)
    food_amounts = zeros(Int, dimensions)
    for food_counter in 1:number_food
        x = rand(1:dimensions[1])
        y = rand(1:dimensions[2])
        food_locations[x, y] = true
        
        food_amounts[x, y] = rand(1:max_food)
    end
    
    properties = (nest_locations = nest_locations, food_locations = food_locations, 
        food_amounts = food_amounts, food_collected = food_collected, 
        pheremone_trails = pheremone_trails, diffusion_rate = diffusion_rate,
        ant_pheremone_amount = ant_pheremone_amount)    
    space = GridSpace(dimensions, periodic = false)
    
    model = UnremovableABM(AntAgent, space; properties, rng, scheduler = Schedulers.Randomly())
    
    for num_nest in 1:number_nests
        for n in 1:number_ants
            agent = AntAgent(n * num_nest, nest_locations[num_nest], num_nest, false, Stack{Tuple}(), (1, 0), 0, 0, 0)
            add_agent_pos!(agent, model)
        end    
    end 
    
    return model
end

model = initialize_ants()

UnremovableABM with 50 agents of type AntAgent
 space: GridSpace with size (64, 64), metric=chebyshev, periodic=false
 scheduler: Agents.Schedulers.Randomly
 properties: nest_locations, food_locations, food_amounts, food_collected, pheremone_trails, diffusion_rate

In [5]:
model.nest_locations


Dict{Any, Any} with 1 entry:
  1 => (14, 38)

In [ ]:
function is_on_pheremone_trail(agent, model)
    return model.pheremone_trails[agent.colony][agent.pos[1], agent.pos[2]] > 10
end

In [6]:
function find_pheremone_direction(agent, model)
    #returns 2 values, 
    min_value, max_value = 500, -500
    min_pos = agent.pos
    max_pos = agent.pos
    for i in range(-1, 1, 1)
        for j in range(-1, 1, 1)
            if i == 0 and j == 0
               break 
            end
            
            if model.pheremone_trails[ant.colony][agent.pos[1] + i, agent.pos[2] + j] >= max_value
                max_value = model.pheremone_trails[ant.colony][agent.pos[1] + i, agent.pos[2] + j]
                max_pos = (agent.pos[1] + i, agent.pos[2] + j)
            end
            if model.pheremone_trails[ant.colony][agent.pos[1] + i, agent.pos[2] + j] <= min_value
                min_value = model.pheremone_trails[ant.colony][agent.pos[1] + i, agent.pos[2] + j]
                min_pos = (agent.pos[1] + i, agent.pos[2] + j)
            end
        end
    end
    return min_pos, max_pos
end

find_pheremone (generic function with 1 method)

In [ ]:
function drop_pheremone(agent, model)
    model.pheremone_trails[agent.colony][agent.pos[1], agent.pos[2]] += model.ant_pheremone_amount
end

In [ ]:
function pickup_food(agent, model)
    agent.has_food = true
    model.food_amounts[agent.pos] -= 1
    agent.food_picked_up += 1
    if model.food_amounts[agent.pos] <= 0 
        model.food_locations[agent.pos] = false
    end
end

In [ ]:
function drop_food(agent, model)
    model.food_collected[agent.colony] += 1
    agent.food_returned_nest += 1
    agent.has_food = false
    empty!(agent.path_home)
end

In [21]:
function agent_step!(agent, model)
    if agent.has_food
        if agent.pos == model.nest_locations[agent.colony]
            # If back at the nest, deposit the food. And then set out again
            drop_food(agent, model)
        elseif is_on_pheremone_trail(agent, model)
            drop_pheremone(agent, model)
            min, max = find_pheremone_direction(agent, model)
            move_agent!(agent, max, model)
        else
            # No trail, so walk to the reverse of the path to the nest
            drop_pheremone(agent, model)
            move_agent!(agent, pop!(agent.path_home), model)      
        end
    else
        # Agent does not have food
        push!(agent.path_home, agent.pos)
        @show agent.pos
        if model.food_locations[agent.pos[1], agent.pos[2]]
            # Found a morsel of food. 
            pickup_food(agent, model)
        elseif is_on_pheremone_trail(agent, model)
            min, max = find_pheremone_direction(agent, model)
            move_agent!(agent, min, model)
        else
            # Randomly search, TODO but do not go back on your last set
            randomwalk!(agent, model, 1)
            @show agent.pos
        end
    end
end

agent_step! (generic function with 1 method)

In [24]:
function model_step!(model)
    for colony in range(1, length(model.nest_locations))
        model.pheremone_trails[colony] *= model.diffusion_rate / 100
    end
end

model_step! (generic function with 1 method)

In [25]:
steps = 100
count_food_collected(model) = model.food_collected[1]
adata = []
mdata = [count_food_collected]
adf, mdf = run!(model, agent_step!, model_step!, steps; adata, mdata)

agent.pos = (15, 39)
agent.pos = (15, 39)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (15, 38)
agent.pos = (16, 38)
agent.pos = (13, 40)
agent.pos = (13, 39)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (14, 39)
agent.pos = (13, 40)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (15, 38)
agent.pos = (17, 38)
agent.pos = (17, 38)
agent.pos = (14, 38)
agent.pos = (14, 38)
agent.pos = (16, 39)
agent.pos = (

(5050×2 DataFrame
  Row │ step   id    
      │ Int64  Int64 
──────┼──────────────
    1 │     0      1
    2 │     0      2
    3 │     0      3
    4 │     0      4
    5 │     0      5
    6 │     0      6
    7 │     0      7
    8 │     0      8
    9 │     0      9
   10 │     0     10
   11 │     0     11
  ⋮   │   ⋮      ⋮
 5041 │   100     41
 5042 │   100     42
 5043 │   100     43
 5044 │   100     44
 5045 │   100     45
 5046 │   100     46
 5047 │   100     47
 5048 │   100     48
 5049 │   100     49
 5050 │   100     50
    5029 rows omitted, 101×2 DataFrame
 Row │ step   count_food_collected 
     │ Int64  Int64                
─────┼─────────────────────────────
   1 │     0                     0
   2 │     1                     0
   3 │     2                     0
   4 │     3                     0
   5 │     4                     0
   6 │     5                     0
   7 │     6                     0
   8 │     7                     0
   9 │     8                 